In [5]:
from IPython.display import clear_output, display, HTML
from ipywidgets import widgets, Output, VBox, HBox
import plotly.express as px
import pandas as pd

# Load the dataset
df = pd.read_excel('FloodArchive.xlsx')
df['Year'] = pd.to_datetime(df['Began']).dt.year

# Display footer content
html_content1 = """
<center style="color:black; font-size: 150%;"> World Wide Floods Dashboard </center>
"""
html_content2 = """
<center><img src="Dartmouth-Boulder.png"/></center>
<center><b> Global Active Archive of Large Flood Events, 1985-Present. Dartmouth Flood Observatory, University of Colorado, USA.</b></center>
"""

# Define the widget
year_widget1 = widgets.IntRangeSlider(value=[2015, 2020], min=1985, max=2025, step=1, description='Years:')

# Create Output widgets
output_geo = Output()
output_bar1 = Output()
output_bar2 = Output()
output_death = Output()

# Function to create the first barchart
def bar_plot_causes(selected_years):
    causes = df[df['Year'].between(selected_years[0], selected_years[1])].groupby(by=['MainCause']).size().reset_index(name='counts')
    causes = causes.sort_values(['counts'], ascending=False)[:10]
    fig = px.bar(data_frame=causes, y='MainCause', x='counts')
    return fig

# Function to create the second barchart
def bar_plot_causes2(selected_years):
    countries = df[df["Year"].between(selected_years[0], selected_years[1])].groupby(by=["Country"])["Dead"].sum().reset_index(name="counts")
    countries = countries.sort_values(['counts'],ascending=False)[:10]
    fig = px.bar(data_frame=countries, x="Country", y="counts")
    return fig

# Function to create the geo plot
def geo_plot(selected_years):
    fig = px.scatter_geo(
        df[df['Year'].between(selected_years[0], selected_years[1])],
        lat='lat',
        lon='long',
        hover_name='Country',
        size='Dead',
        projection='natural earth',
    )
    return fig

# Function to calculate total deaths
def calculate_period_death(selected_years):
    total_death = df[df["Year"].between(selected_years[0], selected_years[1])]["Dead"].sum()
    return widgets.HTML(value=f'<h3>Flooding Caused <i>{total_death} Total Deaths</i> for the selected period</h3>')

# Function to update plots based on the selected year range
def update_plots(change):
    selected_years = change['new']
    
    with output_geo:
        output_geo.clear_output(wait=True)
        display(geo_plot(selected_years))

    with output_bar1:
        output_bar1.clear_output(wait=True)
        display(bar_plot_causes(selected_years))

    with output_bar2:
        output_bar2.clear_output(wait=True)
        fig = bar_plot_causes2(selected_years)
        fig.show()

    with output_death:
        output_death.clear_output(wait=True)
        display(calculate_period_death(selected_years))

# Attach the update_plots function to the widget's value change event
year_widget1.observe(update_plots, names='value')

# Initial display of plots
display(HTML(html_content1))
display(year_widget1)
display(VBox([HBox([output_geo, output_bar1]), HBox([output_bar2, output_death])]))
#display(output_geo)
#display(output_bar1)
#display(output_bar2)
#display(output_death)
display(HTML(html_content2))
#update_plots({'new': year_widget1.value})


IntRangeSlider(value=(2015, 2020), description='Years:', max=2025, min=1985)

Output()

Output()

Output()

Output()